In [2]:
!pip install transformers==4.50.0

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 48.5 MB/s  0:00:00eta 0:00:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.2
    Uninstalling transformers-4.53.2:
      Successfully uninstalled transformers-4.53.2


In [ ]:
!nvidia-smi

In [2]:
import os
print(os.getcwd())
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

/workspace/Approach1


In [4]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [1]:
import transformers
print(transformers.__version__)

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4.50.0


In [2]:
without_punct_model_name = "thenlpresearcher/iitb-en-indic-without-punct"
with_punct_model_name = "thenlpresearcher/iitb-en-indic-only-punct"
combined_punct_model_name = "thenlpresearcher/iitb_en_indic_robust_punctuation_model"

In [3]:
from datasets import load_dataset
raw_datasets = load_dataset("thenlpresearcher/test_data_marathi")

In [4]:
# -------------------- LOAD DATA --------------------
src_sentences = raw_datasets['test']["sent_written"]
ref_gt     = raw_datasets['test']["gt_marathi"]
ref_gem    = raw_datasets['test']["gemini_out"]
ref_cfilt  = raw_datasets['test']["cfilt_out"]

### Original

In [5]:
model_name = "ai4bharat/indictrans2-en-indic-dist-200M"

In [6]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from IndicTransToolkit.processor import IndicProcessor
from tqdm import tqdm
# recommended to run this on a gpu with flash_attn installed
# don't set attn_implemetation if you don't have flash_attn
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

src_lang, tgt_lang = "eng_Latn", "mar_Deva"
tokenizer_name =  "ai4bharat/indictrans2-en-indic-dist-200M"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, trust_remote_code=True)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name, 
    trust_remote_code=True, 
    torch_dtype=torch.float16, # performance might slightly vary for bfloat16
    attn_implementation="flash_attention_2"
).to(DEVICE)

ip = IndicProcessor(inference=True)

input_sentences = src_sentences

import torch

def batch_translate(
    input_sentences,
    src_lang,
    tgt_lang,
    model,
    tokenizer,
    ip,
    device="cuda",
    batch_size=16,
):
    all_translations = []

    for i in tqdm(range(0, len(input_sentences), batch_size)):
        batch = input_sentences[i : i + batch_size]

        # Preprocess (handles entity mapping etc.)
        batch = ip.preprocess_batch(
            batch,
            src_lang=src_lang,
            tgt_lang=tgt_lang
        )

        # Tokenize on device
        inputs = tokenizer(
            batch,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        ).to(device)

        # Generate translations
        with torch.no_grad():
            generated = model.generate(
                **inputs,
                use_cache=True,
                min_length=0,
                max_length=256,
                num_beams=5,
                num_return_sequences=1,
            )

        # Decode
        decoded = tokenizer.batch_decode(
            generated,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True,
        )

        # Postprocess (restore entities)
        decoded = ip.postprocess_batch(decoded, lang=tgt_lang)

        all_translations.extend(decoded)

    return all_translations

translations = batch_translate(
    input_sentences,
    src_lang="eng_Latn",
    tgt_lang="mar_Deva",
    model=model,
    tokenizer=tokenizer,
    ip=ip,
    device=DEVICE,
    batch_size=32
)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

eng_Latn: Chanting the choir raised the volume as the celebrant intoned the prayer.
mar_Deva: उत्सवी गायकांनी प्रार्थनेचा उच्चार केल्याने गायकवृंदाचा जप केल्याने ध्वनीमुद्रात वाढ झाली..........................................................................................................................................................................................................................................
eng_Latn: A six-month-old calf was submitted for examination, showing lameness in all four legs which had been present since soon after birth.
mar_Deva: जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पायांमध्ये लंगडेपणा दर्शविणाऱ्या सहा महिन्यांच्या वासराला तपासणीसाठी सादर करण्यात आले....................................................................................................................................................................................................................................
eng_Latn: Planning authorities should provide alternative locations for s

In [7]:
import re
def remove_prefix(translations, prefix):
    ans = []
    for t in translations:
        t = t.strip()
        if t.startswith(prefix):
            t = t[len(prefix):]
        t = re.sub(r'\.+', '.', t)
        ans.append(t)
    return ans

translations = remove_prefix(translations, "mar_Deva eng_Latn ")
print(translations[0])

उत्सवी गायकांनी प्रार्थनेचा उच्चार केल्याने गायकवृंदाचा जप केल्याने ध्वनीमुद्रात वाढ झाली.


In [8]:
mode = "original"

In [9]:
import pandas as pd
from evaluate import load

# -------------------- SAVE OUTPUTS --------------------
results_df = pd.DataFrame({
    "src": src_sentences,
    "prediction": translations,
    "gt": ref_gt,
    "gemini": ref_gem,
    "cfilt": ref_cfilt
})

results_df.to_csv(f"{mode}_outputs.csv", index=False)
print(f"✔ Saved predictions to {mode}_outputs.csv")

# -------------------- METRICS --------------------
bleu = load("sacrebleu")
chrf = load("chrf")

def compute_scores(preds, ref1, ref2, ref3):
    """
    Compute BLEU and chrF++ scores using all three references for each sentence.
    """
    references = [[r1, r2, r3] for r1, r2, r3 in zip(ref1, ref2, ref3)]  # sacrebleu format
    bleu_score = bleu.compute(predictions=preds, references=references)["score"]
    chrf_score = chrf.compute(predictions=preds, references=references)["score"]
    return bleu_score, chrf_score

bleu_score, chrf_score = compute_scores(translations, ref_gt, ref_gem, ref_cfilt)

# Determine best reference per metric (based on BLEU)
all_scores = {
    "GT":    bleu.compute(predictions=translations, references=[[r] for r in ref_gt])["score"],
    "Gemini": bleu.compute(predictions=translations, references=[[r] for r in ref_gem])["score"],
    "CFILT":  bleu.compute(predictions=translations, references=[[r] for r in ref_cfilt])["score"]
}

best_ref = max(all_scores, key=all_scores.get)

print("\n===== FINAL METRICS =====")
print(f"All references combined → BLEU: {bleu_score:.2f}, chrF++: {chrf_score:.2f}")
print(f"GT Marathi → BLEU: {all_scores['GT']:.2f}")
print(f"Gemini    → BLEU: {all_scores['Gemini']:.2f}")
print(f"CFILT     → BLEU: {all_scores['CFILT']:.2f}")
print(f"\n🎯 BEST REFERENCE = {best_ref} (by highest BLEU)")

# -------------------- SAVE METRICS --------------------
with open(f"punct_{mode}_indictrans2_eval_metrics.txt", "w") as f:
    f.write(f"All references combined → BLEU {bleu_score:.2f}, chrF++ {chrf_score:.2f}\n")
    f.write(f"GT    BLEU {all_scores['GT']:.2f}\n")
    f.write(f"Gem   BLEU {all_scores['Gemini']:.2f}\n")
    f.write(f"CFILT BLEU {all_scores['CFILT']:.2f}\n")
    f.write(f"\nBEST REFERENCE = {best_ref}\n")

print(f"Metrics written to punct_{mode}_baseline_outputs_eval_metrics.txt")

✔ Saved predictions to original_outputs.csv

===== FINAL METRICS =====
All references combined → BLEU: 69.09, chrF++: 84.82
GT Marathi → BLEU: 57.12
Gemini    → BLEU: 21.83
CFILT     → BLEU: 51.14

🎯 BEST REFERENCE = GT (by highest BLEU)
Metrics written to punct_original_baseline_outputs_eval_metrics.txt


### Model Fine-tuned on data without punctuation

In [5]:
model_name = without_punct_model_name

In [6]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from IndicTransToolkit.processor import IndicProcessor
# recommended to run this on a gpu with flash_attn installed
# don't set attn_implemetation if you don't have flash_attn
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

src_lang, tgt_lang = "eng_Latn", "mar_Deva"
tokenizer_name =  "ai4bharat/indictrans2-en-indic-dist-200M"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, trust_remote_code=True)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name, 
    trust_remote_code=True, 
    torch_dtype=torch.float16, # performance might slightly vary for bfloat16
    attn_implementation="flash_attention_2"
).to(DEVICE)

ip = IndicProcessor(inference=True)

input_sentences = src_sentences

batch = ip.preprocess_batch(input_sentences, src_lang=src_lang, tgt_lang=tgt_lang)

# Tokenize the sentences and generate input encodings
inputs = tokenizer(
    batch,
    truncation=True,
    padding="longest",
    return_tensors="pt",
    return_attention_mask=True,
).to(DEVICE)

# Generate translations using the model
with torch.no_grad():
    generated_tokens = model.generate(
        **inputs,
        use_cache=True,
        min_length=0,
        max_length=256,
        num_beams=5,
        num_return_sequences=1,
    )

# Decode the generated tokens into text
generated_tokens = tokenizer.batch_decode(
    generated_tokens,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=True,
)

# Postprocess the translations, including entity replacement
translations = ip.postprocess_batch(generated_tokens, lang=tgt_lang)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")


eng_Latn: Chanting the choir raised the volume as the celebrant intoned the prayer.
mar_Deva: mar_Deva eng_Latn उत्सव साजरा करणार\u093C्याने प्रार्थनेचा उच्चार करताच गायकवृंदाच्या पठणाने आवाज वाढवला.
eng_Latn: A six-month-old calf was submitted for examination, showing lameness in all four legs which had been present since soon after birth.
mar_Deva: mar_Deva eng_Latn जन्मानंतर लगेचच अस्तित्वात असलेल्या सर्व चारही पायांमध्ये लंगडेपणा दर्शविणारी सहा महिन्यांची एक वासर तपासणीसाठी सादर करण्यात आली.
eng_Latn: Planning authorities should provide alternative locations for small businesses which are or would be offensive in a residential area.
mar_Deva: mar_Deva eng_Latn नियोजन प्राधिकरणांनी छोट्या व्यवसायांसाठी पर्यायी ठिकाणे पुरवली पाहिजेत जी निवासी क्षेत्रात आक्षेपार्ह आहेत किंवा असतील.
eng_Latn: As the machine develops the forms we use to record data from past projects will be amended.
mar_Deva: mar_Deva eng_Latn जसजसे मशीन विकसित होते, तसतसे आपण मागील प्रकल्पांमधील डेटा रेकॉर्ड करण्यासाठ

In [5]:
def remove_prefix(translations, prefix):
    ans = []
    for t in translations:
        t = t.strip()
        if t.startswith(prefix):
            t = t[len(prefix):]
        ans.append(t)
    return ans

translations = remove_prefix(translations, "mar_Deva eng_Latn ")
print(translations[0])

उत्सव साजरा करणार\u093C्याने प्रार्थना केली तेव्हा गायकवृंदाचा जप केल्याने आवाज वाढला.


In [6]:
mode = "without"

In [9]:
import pandas as pd
from evaluate import load

# -------------------- SAVE OUTPUTS --------------------
results_df = pd.DataFrame({
    "src": src_sentences,
    "prediction": translations,
    "gt": ref_gt,
    "gemini": ref_gem,
    "cfilt": ref_cfilt
})

results_df.to_csv(f"{mode}_outputs.csv", index=False)
print(f"✔ Saved predictions to {mode}_outputs.csv")

# -------------------- METRICS --------------------
bleu = load("sacrebleu")
chrf = load("chrf")

def compute_scores(preds, ref1, ref2, ref3):
    """
    Compute BLEU and chrF++ scores using all three references for each sentence.
    """
    references = [[r1, r2, r3] for r1, r2, r3 in zip(ref1, ref2, ref3)]  # sacrebleu format
    bleu_score = bleu.compute(predictions=preds, references=references)["score"]
    chrf_score = chrf.compute(predictions=preds, references=references)["score"]
    return bleu_score, chrf_score

bleu_score, chrf_score = compute_scores(translations, ref_gt, ref_gem, ref_cfilt)

# Determine best reference per metric (based on BLEU)
all_scores = {
    "GT":    bleu.compute(predictions=translations, references=[[r] for r in ref_gt])["score"],
    "Gemini": bleu.compute(predictions=translations, references=[[r] for r in ref_gem])["score"],
    "CFILT":  bleu.compute(predictions=translations, references=[[r] for r in ref_cfilt])["score"]
}

best_ref = max(all_scores, key=all_scores.get)

print("\n===== FINAL METRICS =====")
print(f"All references combined → BLEU: {bleu_score:.2f}, chrF++: {chrf_score:.2f}")
print(f"GT Marathi → BLEU: {all_scores['GT']:.2f}")
print(f"Gemini    → BLEU: {all_scores['Gemini']:.2f}")
print(f"CFILT     → BLEU: {all_scores['CFILT']:.2f}")
print(f"\n🎯 BEST REFERENCE = {best_ref} (by highest BLEU)")

# -------------------- SAVE METRICS --------------------
with open(f"punct_{mode}_indictrans2_eval_metrics.txt", "w") as f:
    f.write(f"All references combined → BLEU {bleu_score:.2f}, chrF++ {chrf_score:.2f}\n")
    f.write(f"GT    BLEU {all_scores['GT']:.2f}\n")
    f.write(f"Gem   BLEU {all_scores['Gemini']:.2f}\n")
    f.write(f"CFILT BLEU {all_scores['CFILT']:.2f}\n")
    f.write(f"\nBEST REFERENCE = {best_ref}\n")

print(f"Metrics written to punct_{mode}_baseline_outputs_eval_metrics.txt")

✔ Saved predictions to combined_outputs.csv

===== FINAL METRICS =====
All references combined → BLEU: 60.76, chrF++: 80.54
GT Marathi → BLEU: 42.34
Gemini    → BLEU: 21.21
CFILT     → BLEU: 49.06

🎯 BEST REFERENCE = CFILT (by highest BLEU)
Metrics written to punct_combined_baseline_outputs_eval_metrics.txt


In [4]:
model_name = without_punct_model_name

In [5]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from IndicTransToolkit.processor import IndicProcessor
# recommended to run this on a gpu with flash_attn installed
# don't set attn_implemetation if you don't have flash_attn
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

src_lang, tgt_lang = "eng_Latn", "mar_Deva"
tokenizer_name =  "ai4bharat/indictrans2-en-indic-dist-200M"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, trust_remote_code=True)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name, 
    trust_remote_code=True, 
    torch_dtype=torch.float16, # performance might slightly vary for bfloat16
    attn_implementation="flash_attention_2"
).to(DEVICE)

ip = IndicProcessor(inference=True)

input_sentences = src_sentences

batch = ip.preprocess_batch(input_sentences, src_lang=src_lang, tgt_lang=tgt_lang)

# Tokenize the sentences and generate input encodings
inputs = tokenizer(
    batch,
    truncation=True,
    padding="longest",
    return_tensors="pt",
    return_attention_mask=True,
).to(DEVICE)

# Generate translations using the model
with torch.no_grad():
    generated_tokens = model.generate(
        **inputs,
        use_cache=True,
        min_length=0,
        max_length=256,
        num_beams=5,
        num_return_sequences=1,
    )

# Decode the generated tokens into text
generated_tokens = tokenizer.batch_decode(
    generated_tokens,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=True,
)

# Postprocess the translations, including entity replacement
translations = ip.postprocess_batch(generated_tokens, lang=tgt_lang)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")


eng_Latn: Chanting the choir raised the volume as the celebrant intoned the prayer.
mar_Deva: mar_Deva eng_Latn उत्सव साजरा करणार\u093C्याने प्रार्थनेचा उच्चार करताच गायकवृंदाच्या पठणाने आवाज वाढवला.
eng_Latn: A six-month-old calf was submitted for examination, showing lameness in all four legs which had been present since soon after birth.
mar_Deva: mar_Deva eng_Latn जन्मानंतर लगेचच अस्तित्वात असलेल्या सर्व चारही पायांमध्ये लंगडेपणा दर्शविणारी सहा महिन्यांची एक वासर तपासणीसाठी सादर करण्यात आली.
eng_Latn: Planning authorities should provide alternative locations for small businesses which are or would be offensive in a residential area.
mar_Deva: mar_Deva eng_Latn नियोजन प्राधिकरणांनी छोट्या व्यवसायांसाठी पर्यायी ठिकाणे पुरवली पाहिजेत जी निवासी क्षेत्रात आक्षेपार्ह आहेत किंवा असतील.
eng_Latn: As the machine develops the forms we use to record data from past projects will be amended.
mar_Deva: mar_Deva eng_Latn जसजसे मशीन विकसित होते, तसतसे आपण मागील प्रकल्पांमधील डेटा रेकॉर्ड करण्यासाठ

In [6]:
def remove_prefix(translations, prefix):
    ans = []
    for t in translations:
        t = t.strip()
        if t.startswith(prefix):
            t = t[len(prefix):]
        ans.append(t)
    return ans

translations = remove_prefix(translations, "mar_Deva eng_Latn ")
print(translations[0])

उत्सव साजरा करणार\u093C्याने प्रार्थनेचा उच्चार करताच गायकवृंदाच्या पठणाने आवाज वाढवला.


In [7]:
mode = "without"

In [8]:
import pandas as pd
from evaluate import load

# -------------------- SAVE OUTPUTS --------------------
results_df = pd.DataFrame({
    "src": src_sentences,
    "prediction": translations,
    "gt": ref_gt,
    "gemini": ref_gem,
    "cfilt": ref_cfilt
})

results_df.to_csv(f"{mode}_outputs.csv", index=False)
print(f"✔ Saved predictions to {mode}_outputs.csv")

# -------------------- METRICS --------------------
bleu = load("sacrebleu")
chrf = load("chrf")

def compute_scores(preds, ref1, ref2, ref3):
    """
    Compute BLEU and chrF++ scores using all three references for each sentence.
    """
    references = [[r1, r2, r3] for r1, r2, r3 in zip(ref1, ref2, ref3)]  # sacrebleu format
    bleu_score = bleu.compute(predictions=preds, references=references)["score"]
    chrf_score = chrf.compute(predictions=preds, references=references)["score"]
    return bleu_score, chrf_score

bleu_score, chrf_score = compute_scores(translations, ref_gt, ref_gem, ref_cfilt)

# Determine best reference per metric (based on BLEU)
all_scores = {
    "GT":    bleu.compute(predictions=translations, references=[[r] for r in ref_gt])["score"],
    "Gemini": bleu.compute(predictions=translations, references=[[r] for r in ref_gem])["score"],
    "CFILT":  bleu.compute(predictions=translations, references=[[r] for r in ref_cfilt])["score"]
}

best_ref = max(all_scores, key=all_scores.get)

print("\n===== FINAL METRICS =====")
print(f"All references combined → BLEU: {bleu_score:.2f}, chrF++: {chrf_score:.2f}")
print(f"GT Marathi → BLEU: {all_scores['GT']:.2f}")
print(f"Gemini    → BLEU: {all_scores['Gemini']:.2f}")
print(f"CFILT     → BLEU: {all_scores['CFILT']:.2f}")
print(f"\n🎯 BEST REFERENCE = {best_ref} (by highest BLEU)")

# -------------------- SAVE METRICS --------------------
with open(f"punct_{mode}_indictrans2_eval_metrics.txt", "w") as f:
    f.write(f"All references combined → BLEU {bleu_score:.2f}, chrF++ {chrf_score:.2f}\n")
    f.write(f"GT    BLEU {all_scores['GT']:.2f}\n")
    f.write(f"Gem   BLEU {all_scores['Gemini']:.2f}\n")
    f.write(f"CFILT BLEU {all_scores['CFILT']:.2f}\n")
    f.write(f"\nBEST REFERENCE = {best_ref}\n")

print(f"Metrics written to punct_{mode}_baseline_outputs_eval_metrics.txt")

✔ Saved predictions to without_outputs.csv

===== FINAL METRICS =====
All references combined → BLEU: 63.10, chrF++: 80.97
GT Marathi → BLEU: 43.28
Gemini    → BLEU: 24.66
CFILT     → BLEU: 51.86

🎯 BEST REFERENCE = CFILT (by highest BLEU)
Metrics written to punct_without_baseline_outputs_eval_metrics.txt


In [9]:
model_name = with_punct_model_name

In [10]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from IndicTransToolkit.processor import IndicProcessor
# recommended to run this on a gpu with flash_attn installed
# don't set attn_implemetation if you don't have flash_attn
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

src_lang, tgt_lang = "eng_Latn", "mar_Deva"
tokenizer_name =  "ai4bharat/indictrans2-en-indic-dist-200M"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, trust_remote_code=True)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name, 
    trust_remote_code=True, 
    torch_dtype=torch.float16, # performance might slightly vary for bfloat16
    attn_implementation="flash_attention_2"
).to(DEVICE)

ip = IndicProcessor(inference=True)

input_sentences = src_sentences

batch = ip.preprocess_batch(input_sentences, src_lang=src_lang, tgt_lang=tgt_lang)

# Tokenize the sentences and generate input encodings
inputs = tokenizer(
    batch,
    truncation=True,
    padding="longest",
    return_tensors="pt",
    return_attention_mask=True,
).to(DEVICE)

# Generate translations using the model
with torch.no_grad():
    generated_tokens = model.generate(
        **inputs,
        use_cache=True,
        min_length=0,
        max_length=256,
        num_beams=5,
        num_return_sequences=1,
    )

# Decode the generated tokens into text
generated_tokens = tokenizer.batch_decode(
    generated_tokens,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=True,
)

# Postprocess the translations, including entity replacement
translations = ip.postprocess_batch(generated_tokens, lang=tgt_lang)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")


eng_Latn: Chanting the choir raised the volume as the celebrant intoned the prayer.
mar_Deva: mar_Deva eng_Latn उत्सव साजरा करणार\u093C्याने प्रार्थना केली तेव्हा गायकवृंदाचा जप केल्याने आवाज वाढला.
eng_Latn: A six-month-old calf was submitted for examination, showing lameness in all four legs which had been present since soon after birth.
mar_Deva: mar_Deva eng_Latn जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पायांमध्ये लंगडेपणा दिसून येत असलेल्या सहा महिन्यांच्या वासूला तपासणीसाठी सादर करण्यात आले.
eng_Latn: Planning authorities should provide alternative locations for small businesses which are or would be offensive in a residential area.
mar_Deva: mar_Deva eng_Latn नियोजन प्राधिकरणांनी निवासी क्षेत्रात आक्षेपार्ह असलेल्या किंवा असणार\u093C्या लहान व्यवसायांसाठी पर्यायी ठिकाणे पुरवली पाहिजेत.
eng_Latn: As the machine develops the forms we use to record data from past projects will be amended.
mar_Deva: mar_Deva eng_Latn जसजसे मशीन विकसित होईल तसतसे आम्ही मागील प्रकल्पांमधील डेटा रेकॉर

In [11]:
def remove_prefix(translations, prefix):
    ans = []
    for t in translations:
        t = t.strip()
        if t.startswith(prefix):
            t = t[len(prefix):]
        ans.append(t)
    return ans

translations = remove_prefix(translations, "mar_Deva eng_Latn ")
print(translations[0])

उत्सव साजरा करणार\u093C्याने प्रार्थना केली तेव्हा गायकवृंदाचा जप केल्याने आवाज वाढला.


In [12]:
mode = "only"

In [13]:
import pandas as pd
from evaluate import load

# -------------------- SAVE OUTPUTS --------------------
results_df = pd.DataFrame({
    "src": src_sentences,
    "prediction": translations,
    "gt": ref_gt,
    "gemini": ref_gem,
    "cfilt": ref_cfilt
})

results_df.to_csv(f"{mode}_outputs.csv", index=False)
print(f"✔ Saved predictions to {mode}_outputs.csv")

# -------------------- METRICS --------------------
bleu = load("sacrebleu")
chrf = load("chrf")

def compute_scores(preds, ref1, ref2, ref3):
    """
    Compute BLEU and chrF++ scores using all three references for each sentence.
    """
    references = [[r1, r2, r3] for r1, r2, r3 in zip(ref1, ref2, ref3)]  # sacrebleu format
    bleu_score = bleu.compute(predictions=preds, references=references)["score"]
    chrf_score = chrf.compute(predictions=preds, references=references)["score"]
    return bleu_score, chrf_score

bleu_score, chrf_score = compute_scores(translations, ref_gt, ref_gem, ref_cfilt)

# Determine best reference per metric (based on BLEU)
all_scores = {
    "GT":    bleu.compute(predictions=translations, references=[[r] for r in ref_gt])["score"],
    "Gemini": bleu.compute(predictions=translations, references=[[r] for r in ref_gem])["score"],
    "CFILT":  bleu.compute(predictions=translations, references=[[r] for r in ref_cfilt])["score"]
}

best_ref = max(all_scores, key=all_scores.get)

print("\n===== FINAL METRICS =====")
print(f"All references combined → BLEU: {bleu_score:.2f}, chrF++: {chrf_score:.2f}")
print(f"GT Marathi → BLEU: {all_scores['GT']:.2f}")
print(f"Gemini    → BLEU: {all_scores['Gemini']:.2f}")
print(f"CFILT     → BLEU: {all_scores['CFILT']:.2f}")
print(f"\n🎯 BEST REFERENCE = {best_ref} (by highest BLEU)")

# -------------------- SAVE METRICS --------------------
with open(f"punct_{mode}_indictrans2_eval_metrics.txt", "w") as f:
    f.write(f"All references combined → BLEU {bleu_score:.2f}, chrF++ {chrf_score:.2f}\n")
    f.write(f"GT    BLEU {all_scores['GT']:.2f}\n")
    f.write(f"Gem   BLEU {all_scores['Gemini']:.2f}\n")
    f.write(f"CFILT BLEU {all_scores['CFILT']:.2f}\n")
    f.write(f"\nBEST REFERENCE = {best_ref}\n")

print(f"Metrics written to punct_{mode}_baseline_outputs_eval_metrics.txt")

✔ Saved predictions to only_outputs.csv

===== FINAL METRICS =====
All references combined → BLEU: 60.76, chrF++: 80.54
GT Marathi → BLEU: 42.34
Gemini    → BLEU: 21.21
CFILT     → BLEU: 49.06

🎯 BEST REFERENCE = CFILT (by highest BLEU)
Metrics written to punct_only_baseline_outputs_eval_metrics.txt


In [14]:
model_name = combined_punct_model_name

In [ ]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from IndicTransToolkit.processor import IndicProcessor
# recommended to run this on a gpu with flash_attn installed
# don't set attn_implemetation if you don't have flash_attn
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

src_lang, tgt_lang = "eng_Latn", "mar_Deva"
tokenizer_name =  "ai4bharat/indictrans2-en-indic-dist-200M"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, trust_remote_code=True)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name, 
    trust_remote_code=True, 
    torch_dtype=torch.float16, # performance might slightly vary for bfloat16
    attn_implementation="flash_attention_2"
).to(DEVICE)

ip = IndicProcessor(inference=True)

input_sentences = src_sentences

batch = ip.preprocess_batch(input_sentences, src_lang=src_lang, tgt_lang=tgt_lang)

# Tokenize the sentences and generate input encodings
inputs = tokenizer(
    batch,
    truncation=True,
    padding="longest",
    return_tensors="pt",
    return_attention_mask=True,
).to(DEVICE)

# Generate translations using the model
with torch.no_grad():
    generated_tokens = model.generate(
        **inputs,
        use_cache=True,
        min_length=0,
        max_length=256,
        num_beams=5,
        num_return_sequences=1,
    )

# Decode the generated tokens into text
generated_tokens = tokenizer.batch_decode(
    generated_tokens,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=True,
)

# Postprocess the translations, including entity replacement
translations = ip.postprocess_batch(generated_tokens, lang=tgt_lang)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")


In [16]:
def remove_prefix(translations, prefix):
    ans = []
    for t in translations:
        t = t.strip()
        if t.startswith(prefix):
            t = t[len(prefix):]
        ans.append(t)
    return ans

translations = remove_prefix(translations, "mar_Deva eng_Latn ")
print(translations[0])

उत्सव साजरा करणार\u093C्याने प्रार्थनेचा सूर वाजवल्यामुळे गायकवृंदाचा जप केल्याने आवाज वाढला.


In [17]:
mode = "combined"

In [18]:
import pandas as pd
from evaluate import load

# -------------------- SAVE OUTPUTS --------------------
results_df = pd.DataFrame({
    "src": src_sentences,
    "prediction": translations,
    "gt": ref_gt,
    "gemini": ref_gem,
    "cfilt": ref_cfilt
})

results_df.to_csv(f"{mode}_outputs.csv", index=False)
print(f"✔ Saved predictions to {mode}_outputs.csv")

# -------------------- METRICS --------------------
bleu = load("sacrebleu")
chrf = load("chrf")

def compute_scores(preds, ref1, ref2, ref3):
    """
    Compute BLEU and chrF++ scores using all three references for each sentence.
    """
    references = [[r1, r2, r3] for r1, r2, r3 in zip(ref1, ref2, ref3)]  # sacrebleu format
    bleu_score = bleu.compute(predictions=preds, references=references)["score"]
    chrf_score = chrf.compute(predictions=preds, references=references)["score"]
    return bleu_score, chrf_score

bleu_score, chrf_score = compute_scores(translations, ref_gt, ref_gem, ref_cfilt)

# Determine best reference per metric (based on BLEU)
all_scores = {
    "GT":    bleu.compute(predictions=translations, references=[[r] for r in ref_gt])["score"],
    "Gemini": bleu.compute(predictions=translations, references=[[r] for r in ref_gem])["score"],
    "CFILT":  bleu.compute(predictions=translations, references=[[r] for r in ref_cfilt])["score"]
}

best_ref = max(all_scores, key=all_scores.get)

print("\n===== FINAL METRICS =====")
print(f"All references combined → BLEU: {bleu_score:.2f}, chrF++: {chrf_score:.2f}")
print(f"GT Marathi → BLEU: {all_scores['GT']:.2f}")
print(f"Gemini    → BLEU: {all_scores['Gemini']:.2f}")
print(f"CFILT     → BLEU: {all_scores['CFILT']:.2f}")
print(f"\n🎯 BEST REFERENCE = {best_ref} (by highest BLEU)")

# -------------------- SAVE METRICS --------------------
with open(f"punct_{mode}_indictrans2_eval_metrics.txt", "w") as f:
    f.write(f"All references combined → BLEU {bleu_score:.2f}, chrF++ {chrf_score:.2f}\n")
    f.write(f"GT    BLEU {all_scores['GT']:.2f}\n")
    f.write(f"Gem   BLEU {all_scores['Gemini']:.2f}\n")
    f.write(f"CFILT BLEU {all_scores['CFILT']:.2f}\n")
    f.write(f"\nBEST REFERENCE = {best_ref}\n")

print(f"Metrics written to punct_{mode}_baseline_outputs_eval_metrics.txt")

✔ Saved predictions to combined_outputs.csv

===== FINAL METRICS =====
All references combined → BLEU: 64.73, chrF++: 81.78
GT Marathi → BLEU: 43.30
Gemini    → BLEU: 25.18
CFILT     → BLEU: 54.13

🎯 BEST REFERENCE = CFILT (by highest BLEU)
Metrics written to punct_combined_baseline_outputs_eval_metrics.txt
